In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

228813984/228813984 [==============================] - 3s 0us/step


In [3]:
list_ds = tf.data.Dataset.list_files(str(data_dir / '*/*'), shuffle=False)
image_count = len(list_ds)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [4]:
for f in list_ds.take(5):
  print(f.numpy())

b'/root/.keras/datasets/flower_photos/tulips/8712267813_f7a9be2ec5.jpg'
b'/root/.keras/datasets/flower_photos/dandelion/4713958242_fbcfe9a61b_m.jpg'
b'/root/.keras/datasets/flower_photos/tulips/10995953955_089572caf0.jpg'
b'/root/.keras/datasets/flower_photos/roses/17090993740_fcc8b60b81.jpg'
b'/root/.keras/datasets/flower_photos/sunflowers/22405882322_d4561f8469_n.jpg'


In [26]:
class_names = np.array(['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'])
class_names

array(['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'],
      dtype='<U10')

In [6]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [7]:
len(val_ds)

734

In [8]:
len(train_ds)


2936

In [10]:
batch_size = 32
img_height = 180
img_width = 180

In [29]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [18]:

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

In [13]:
def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [35]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds_maped = train_ds.map(process_path).batch(batch_size)
val_ds_maped = val_ds.map(process_path).batch(batch_size)

In [31]:
next(iter(train_ds_maped))[1]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([4, 2, 1, 1, 3, 1, 4, 4, 1, 4, 0, 2, 1, 1, 2, 3, 3, 2, 0, 3, 3, 3,
       4, 4, 4, 4, 3, 0, 1, 2, 0, 0])>

In [37]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [38]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [39]:
model.fit(
  train_ds_maped,
  validation_data=val_ds_maped,
  epochs=3
)

Epoch 1/3
92/92 [==============================] - 87s 932ms/step - loss: 1.2621 - accuracy: 0.4656 - val_loss: 1.1168 - val_accuracy: 0.5477
Epoch 2/3
92/92 [==============================] - 86s 933ms/step - loss: 0.9586 - accuracy: 0.6274 - val_loss: 1.0737 - val_accuracy: 0.5790
Epoch 3/3
92/92 [==============================] - 85s 928ms/step - loss: 0.7996 - accuracy: 0.6986 - val_loss: 1.0093 - val_accuracy: 0.6240
